## Import des dépendances

In [19]:
import pandas as pd

## Importation du dataset et split en train, validation et test

In [ ]:
df = pd.read_csv('data/diabetes_clean.csv')
df.head()

# Séparer les features (X) de la cible (y)
y_train = df['class']  # ✅ LABELS (0=mort, 1=survie)

# ⚠️ CORRECTION : Exclure 'id' et 'class' des features
# L'ID n'a aucune valeur prédictive médicale !
X_train = df.drop(['class', 'id'], axis=1)  # ✅ FEATURES (sans la cible et sans l'ID)
print(f'Shape X-train: {X_train.shape}, Shape y-train: {y_train.shape}')
print(f'Colonnes dans X_train: {list(X_train.columns)}')

Shape X-train: (416, 17), Shape y-train: (416,)


In [ ]:
# ✅ VÉRIFICATION : S'assurer qu'on n'a pas d'ID dans les features
print("=== VÉRIFICATION DES FEATURES ===")
print(f"Features utilisées pour l'entraînement: {list(X_train.columns)}")
print(f"Nombre de features: {len(X_train.columns)}")

# Vérifier qu'il n'y a pas d'ID
if 'id' in X_train.columns:
    print("❌ ERREUR: L'ID est encore présent dans les features!")
else:
    print("✅ PARFAIT: L'ID a été exclu des features d'entraînement")

# Afficher quelques statistiques
print(f"\nStatistiques des données:")
print(f"- Nombre d'échantillons: {X_train.shape[0]}")
print(f"- Nombre de features: {X_train.shape[1]}")
print(f"- Distribution des classes: {y_train.value_counts().to_dict()}")

## Encodage des variables (Features + Target)

In [21]:
# 🔤 ENCODAGE DES VARIABLES CATÉGORIELLES
# Import 

from sklearn.preprocessing import LabelEncoder


print("\n=== VARIABLES À ENCODER ===")
# Identifier les colonnes non-numériques
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()
print(f"Colonnes catégorielles: {categorical_columns}")

# Afficher les valeurs uniques avant encodage
for col in categorical_columns:
    print(f"\n{col}: {X_train[col].unique()}")

# Encoder chaque colonne catégorielle
label_encoders = {}
X_train_encoded = X_train.copy()

for col in categorical_columns:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train[col])
    label_encoders[col] = le  # Sauvegarder pour plus tard
    
    print(f"\n✅ {col} encodé:")
    print(f"   Avant: {X_train[col].unique()}")
    print(f"   Après: {X_train_encoded[col].unique()}")

print(f"\nRÉSULTAT FINAL X_train :")
print(f"X_train_encoded shape: {X_train_encoded.shape}")
print(f"Types de données:")
print(X_train_encoded.dtypes)

# Vérification : plus de colonnes 'object'
if X_train_encoded.select_dtypes(include=['object']).shape[1] == 0:
    print(f"\n✅ PARFAIT ! Toutes les variables X sont maintenant numériques")
else:
    print(f"\n❌ Il reste des variables non-numériques à traiter dans X")

print(f"\n=== ENCODAGE DE LA VARIABLE CIBLE (y_train) ===")
print(f"y_train avant encodage: {y_train.unique()}")
print(f"Type de y_train: {y_train.dtype}")

# Encoder y_train
le_target = LabelEncoder()
y_train_encoded = le_target.fit_transform(y_train)

print(f"\n✅ y_train encodé:")
print(f"   Avant: {y_train.unique()}")
print(f"   Après: {le_target.classes_} -> {[0, 1]}")
print(f"   Mapping: {dict(zip(le_target.classes_, [0, 1]))}")

print(f"\n=== VÉRIFICATION FINALE ===")
print(f"X_train_encoded shape: {X_train_encoded.shape}, dtype: numériques ✅")
print(f"y_train_encoded shape: {y_train_encoded.shape}, dtype: {y_train_encoded.dtype} ✅")
print(f"\nTous les encodages sont corrects ! Prêt pour l'entraînement du modèle 🚀")

print(f"\nAPERÇU DES DONNÉES ENCODÉES:")
print("X_train_encoded:")
print(X_train_encoded.head())
print(f"\ny_train_encoded (premiers 10):")
print(y_train_encoded[:10])


=== VARIABLES À ENCODER ===
Colonnes catégorielles: ['gender', 'polyuria', 'polydipsia', 'sudden_weight_loss', 'weakness', 'polyphagia', 'genital_thrush', 'visual_blurring', 'itching', 'irritability', 'delayed_healing', 'partial_paresis', 'muscle_stiffness', 'alopecia', 'obesity']

gender: ['Female' 'Male']

polyuria: ['Yes' 'No']

polydipsia: ['No' 'Yes']

sudden_weight_loss: ['Yes' 'No']

weakness: ['Yes' 'No']

polyphagia: ['No' 'Yes']

genital_thrush: ['Yes' 'No']

visual_blurring: ['Yes' 'No']

itching: ['Yes' 'No']

irritability: ['No' 'Yes']

delayed_healing: ['Yes' 'No']

partial_paresis: ['Yes' 'No']

muscle_stiffness: ['Yes' 'No']

alopecia: ['No' 'Yes']

obesity: ['Yes' 'No']

✅ gender encodé:
   Avant: ['Female' 'Male']
   Après: [0 1]

✅ polyuria encodé:
   Avant: ['Yes' 'No']
   Après: [1 0]

✅ polydipsia encodé:
   Avant: ['No' 'Yes']
   Après: [0 1]

✅ sudden_weight_loss encodé:
   Avant: ['Yes' 'No']
   Après: [1 0]

✅ weakness encodé:
   Avant: ['Yes' 'No']
   Après:

## Split en train, validation, test

In [ ]:
# 🚀 ENTRAÎNEMENT DU MODÈLE RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, log_loss
import numpy as np

print("=== ENTRAÎNEMENT DU MODÈLE ===")

# Création et entraînement du modèle
model = RandomForestClassifier(
    random_state=42,
    n_estimators=300,     # Nombre d'arbres
    max_depth=8,         # Profondeur des arbres
    min_samples_split=5,  # Minimum d'échantillons pour diviser
    min_samples_leaf=2    # Minimum d'échantillons par feuille
)

print(f"🏃‍♂️ Entraînement en cours...")
print(f"Features utilisées: {list(X_train_encoded.columns)}")
print(f"Shape des données: {X_train_encoded.shape}")

model.fit(X_train_encoded, y_train_encoded)
# Sauvegarder le modèle CORRIGÉ (sans ID)
import joblib
model_filename = 'Model_diabetes_RF_FIXED.pkl'
joblib.dump(model, model_filename)
print(f"\n💾 Modèle CORRIGÉ sauvegardé: '{model_filename}'")
print(f"✅ Entraînement terminé !")
print(f"📊 Features dans le modèle: {len(X_train_encoded.columns)} colonnes")
print(f"🎯 Features: {list(X_train_encoded.columns)}")

=== ENTRAÎNEMENT DU MODÈLE ===
🏃‍♂️ Entraînement en cours...

💾 Modèle sauvegardé: 'Model_diabetes_RF.pkl'
✅ Entraînement terminé !

💾 Modèle sauvegardé: 'Model_diabetes_RF.pkl'
✅ Entraînement terminé !


In [23]:
data_csv = pd.read_csv("data/test_without_class.csv")
print(data_csv.head())
data_csv_copy = data_csv.copy()
doublons = data_csv_copy.duplicated(keep=False).sum()
print(f"il ya {doublons} doublons dans le dataset")
nans = data_csv_copy.isna().sum()
print(f"il y a {nans} nans dans le dataset")
print(data_csv_copy["Age"].describe())
data_csv_copy.columns = data_csv_copy.columns.str.replace(' ', '_').str.lower()
print(data_csv_copy.columns)

data_csv_copy.to_csv("data/test_without_class_cleaned.csv", index=False)

    ID  Age  Gender Polyuria Polydipsia sudden weight loss weakness  \
0  417   50  Female       No         No                 No      Yes   
1  418   55    Male       No        Yes                 No      Yes   
2  419   67    Male      Yes        Yes                 No      Yes   
3  420   45    Male       No         No                 No       No   
4  421   37    Male       No         No                 No       No   

  Polyphagia Genital thrush visual blurring Itching Irritability  \
0         No             No             Yes     Yes           No   
1         No            Yes              No      No          Yes   
2        Yes            Yes              No     Yes          Yes   
3        Yes            Yes              No      No           No   
4         No             No              No      No           No   

  delayed healing partial paresis muscle stiffness Alopecia Obesity  
0             Yes              No               No      Yes      No  
1             Yes       

## Test du modèle sur données de test

In [24]:
# 📊 CHARGEMENT DES DONNÉES DE TEST
print("=== CHARGEMENT DU FICHIER DE TEST ===")

# Charger le fichier test (sans la colonne 'class')
df_test = pd.read_csv('data/test_without_class_cleaned.csv')
print(f"Données de test chargées: {df_test.shape}")
print(f"Colonnes disponibles: {list(df_test.columns)}")

# Vérifier la structure
print(f"\nAperçu des données de test:")
print(df_test.head())

# IMPORTANT: Garder la colonne 'id' comme feature (même structure que l'entraînement)
test_ids = df_test['id'].copy()  # Sauvegarder les IDs pour plus tard
X_test = df_test  # Garder TOUTES les colonnes y compris 'id'

print(f"\nShape X_test: {X_test.shape}")
print(f"Colonnes dans X_test: {list(X_test.columns)}")
print(f"Types de données dans X_test:")
print(X_test.dtypes)

=== CHARGEMENT DU FICHIER DE TEST ===
Données de test chargées: (104, 17)
Colonnes disponibles: ['id', 'age', 'gender', 'polyuria', 'polydipsia', 'sudden_weight_loss', 'weakness', 'polyphagia', 'genital_thrush', 'visual_blurring', 'itching', 'irritability', 'delayed_healing', 'partial_paresis', 'muscle_stiffness', 'alopecia', 'obesity']

Aperçu des données de test:
    id  age  gender polyuria polydipsia sudden_weight_loss weakness  \
0  417   50  Female       No         No                 No      Yes   
1  418   55    Male       No        Yes                 No      Yes   
2  419   67    Male      Yes        Yes                 No      Yes   
3  420   45    Male       No         No                 No       No   
4  421   37    Male       No         No                 No       No   

  polyphagia genital_thrush visual_blurring itching irritability  \
0         No             No             Yes     Yes           No   
1         No            Yes              No      No          Yes   
2

In [25]:
# 🔤 ENCODAGE DES DONNÉES DE TEST
print("=== ENCODAGE DES DONNÉES DE TEST ===")

# Identifier les colonnes catégorielles dans le test (exclure 'id' qui est numérique)
categorical_columns_test = X_test.select_dtypes(include=['object']).columns.tolist()
print(f"Colonnes catégorielles dans le test: {categorical_columns_test}")

# Copier les données pour encodage
X_test_encoded = X_test.copy()

# Utiliser les mêmes encodeurs que pour l'entraînement
print(f"\nEncodage avec les encodeurs d'entraînement...")
for col in categorical_columns_test:
    if col in label_encoders:
        # Utiliser l'encodeur d'entraînement
        le = label_encoders[col]
        
        # Vérifier si toutes les valeurs du test sont connues
        unknown_values = set(X_test[col].unique()) - set(le.classes_)
        if unknown_values:
            print(f"⚠️ Valeurs inconnues dans {col}: {unknown_values}")
            # Gérer les valeurs inconnues (les remplacer par la valeur la plus fréquente)
            mode_value = X_test[col].mode()[0]
            X_test_encoded[col] = X_test[col].fillna(mode_value)
        
        X_test_encoded[col] = le.transform(X_test_encoded[col])
        print(f"✅ {col} encodé avec l'encodeur d'entraînement")
    else:
        print(f"❌ Pas d'encodeur trouvé pour {col}")

print(f"\nRÉSULTAT ENCODAGE TEST:")
print(f"X_test_encoded shape: {X_test_encoded.shape}")
print(f"Colonnes dans X_test_encoded: {list(X_test_encoded.columns)}")
print(f"Types de données:")
print(X_test_encoded.dtypes)

# Vérification finale
if X_test_encoded.select_dtypes(include=['object']).shape[1] == 0:
    print(f"\n✅ PARFAIT ! Toutes les variables de test sont numériques")
else:
    print(f"\n❌ Il reste des variables non-numériques dans le test")

print(f"\nAperçu des données test encodées:")
print(X_test_encoded.head())

=== ENCODAGE DES DONNÉES DE TEST ===
Colonnes catégorielles dans le test: ['gender', 'polyuria', 'polydipsia', 'sudden_weight_loss', 'weakness', 'polyphagia', 'genital_thrush', 'visual_blurring', 'itching', 'irritability', 'delayed_healing', 'partial_paresis', 'muscle_stiffness', 'alopecia', 'obesity']

Encodage avec les encodeurs d'entraînement...
✅ gender encodé avec l'encodeur d'entraînement
✅ polyuria encodé avec l'encodeur d'entraînement
✅ polydipsia encodé avec l'encodeur d'entraînement
✅ sudden_weight_loss encodé avec l'encodeur d'entraînement
✅ weakness encodé avec l'encodeur d'entraînement
✅ polyphagia encodé avec l'encodeur d'entraînement
✅ genital_thrush encodé avec l'encodeur d'entraînement
✅ visual_blurring encodé avec l'encodeur d'entraînement
✅ itching encodé avec l'encodeur d'entraînement
✅ irritability encodé avec l'encodeur d'entraînement
✅ delayed_healing encodé avec l'encodeur d'entraînement
✅ partial_paresis encodé avec l'encodeur d'entraînement
✅ muscle_stiffness 

In [26]:
# 🔮 PRÉDICTIONS SUR LES DONNÉES DE TEST
print("=== PRÉDICTIONS SUR LE DATASET DE TEST ===")

# Faire les prédictions
y_test_pred = model.predict(X_test_encoded)

print(f"Prédictions effectuées sur {len(y_test_pred)} échantillons")
print(f"Classes prédites: {np.bincount(y_test_pred)}")  # [nombre de 0, nombre de 1]

=== PRÉDICTIONS SUR LE DATASET DE TEST ===
Prédictions effectuées sur 104 échantillons
Classes prédites: [42 62]


In [27]:
# 💾 SAUVEGARDE DES RÉSULTATS DE TEST
print("=== SAUVEGARDE DES RÉSULTATS ===")

# Créer le DataFrame de soumission (format: ID + class en 0/1)
submission_df = pd.DataFrame({
    'ID': test_ids,
    'class': y_test_pred
})

# Sauvegarder le fichier de soumission
output_filename = 'predictions_test_diabetes.csv'
submission_df.to_csv(output_filename, index=False)

print(f"✅ Fichier sauvegardé: {output_filename}")
print(f"Format: ID, class (0/1)")
print(f"Total: {len(submission_df)} prédictions")

=== SAUVEGARDE DES RÉSULTATS ===
✅ Fichier sauvegardé: predictions_test_diabetes.csv
Format: ID, class (0/1)
Total: 104 prédictions
